In [ ]:
#imports

In [5]:
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Flatten, BatchNormalization, Activation, Lambda, MaxPooling1D
from keras.losses import mse, binary_crossentropy, categorical_crossentropy
from keras.utils import plot_model
from keras.optimizers import SGD, adam, RMSprop, Nadam, Adadelta
from tensorflow.keras.models import load_model
from keras import backend as K

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from keras.wrappers.scikit_learn import KerasRegressor

from read_nilmtk import read_data
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
import pandas as pd
import numpy as np

ImportError: No module named read_nilmtk

In [ ]:
#define the AE

In [ ]:
def shallow(optimizer='Nadam', loss='categorical_crossentropy', mid_activation='sigmoid', out_activation='sigmoid'):
    
    input_window = Input(shape=(window_length,))
    

    encoded = Dense(latent_size, activation=mid_activation)(input_window)
    
    decoded = Dense(window_length, activation=out_activation)(encoded)

    encoder = Model(input_window, encoded)
    autoencoder = Model(input_window, decoded)

    autoencoder.compile(optimizer=optimizer, loss=loss)

    return autoencoder, encoder

In [ ]:
#load data

In [ ]:
x_train, x_test, x_train_shifted_8, x_test_shifted_8, v_train, v_test  = read_data(1,'microwave',1)
x_train = np.squeeze(x_train, axis=2)
x_test = np.squeeze(x_test, axis=2)
x_train_shifted_8 = np.squeeze(x_train_shifted_8, axis=2)
x_test_shifted_8 = np.squeeze(x_test_shifted_8, axis=2)
v_train = np.squeeze(v_train, axis=2)
v_test = np.squeeze(v_test, axis=2)


#train_set = np.concatenate((x_train,x_train_shifted_8), axis=0)
train_set = x_train
test_set = x_test
pos_shifted_train_set = x_train_shifted_8
pos_shifted_set = x_test_shifted_8
neg_shifted_set = v_test

In [ ]:
#hyperparameters

In [ ]:
latent_size = 2
window_length=24
optimizer='Nadam'
loss='categorical_crossentropy'
mid_activation='sigmoid'
out_activation='sigmoid'

In [ ]:
#create and train the AE

In [ ]:
autoencoder, encoder = shallow()
autoencoder.fit(train_set, train_set, batch_size=32,
                epochs=200, shuffle=True)

In [ ]:
#get the latent space of the AE

In [ ]:
latent_train = encoder.predict(train_set)
latent_shifted_train = encoder.predict(pos_shifted_train_set)
latent_test = encoder.predict(test_set)
latent_shifted_pos = encoder.predict(pos_shifted_set)

In [ ]:
#visualize the latent space

In [ ]:
if latent_size == 2:
    X,Y = latent_test.T
    XN,YN = latent_shifted_pos.T
    #XNN,YNN = latent_shifted_neg.T
    plt.plot(X, Y, 'go')
    plt.plot(XN, YN, 'ro')
    #plt.plot(XNN, YNN, 'yo')
elif latent_size ==3:
    from mpl_toolkits.mplot3d import axes3d

    data = (latent_test, latent_shifted_pos,latent_shifted_neg)
    colors = ("green", "red", "yellow")
    groups = ("normal", "anomaly", "anomaly")

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax = fig.gca(projection='3d')


    #X,Y,Z = latent_test.T
    #XN,YN,ZN = latent_shifted_pos.T
    for data, color, group in zip(data, colors, groups):
        x, y, z = data.T
        ax.scatter(x, y, z, alpha=0.8, c=color, edgecolors='none', s=30, label=group)
else:
    pass


In [ ]:
#prepare latent data for classification

In [ ]:
def class_preprocess(latent_test, latent_ran_test):
	L = []
	len(L)
	for x in latent_test:
		L.append([x,0])
	for x in latent_ran_test:
		L.append([x,1])

	L = np.array(L)
	np.random.shuffle(L)
	L.shape
	L[0]
	L[1]

	inputs = [x[0] for x in L]
	targets = [x[1] for x in L]

	return inputs, targets

In [ ]:
#train
features_matrix_train, labels_train = class_preprocess(latent_train, latent_shifted_train)

#test
features_matrix_test, labels_test = class_preprocess(latent_test, latent_shifted_pos)

In [ ]:
#create and train the random forest classifier
classifier = RandomForestClassifier(bootstrap= True, min_samples_leaf = 5, n_estimators = 1000, min_samples_split = 8, max_features = 2, max_depth = 80)
classifier.fit(features_matrix_train, labels_train)

In [ ]:
#predict and evaluate
predicted_labels_test = classifier.predict(np.array(features_matrix_test))

print(confusion_matrix(labels_test,predicted_labels_test))
print(classification_report(labels_test,predicted_labels_test))